# tutorials

* https://fastapi.tiangolo.com/tutorial/
* https://fastapi.tiangolo.com/zh/tutorial/query-params/
* https://www.w3schools.cn/fastapi/index.html   

# tools

## PyNest

* 比较新，
* PyNest is a Python framework built on top of FastAPI that follows the modular architecture of NestJS
* https://pythonnest.github.io/PyNest/
* https://github.com/PythonNest/PyNest

# Adds simple SQLAlchemy support to FastAPI
* pip install fastapi-sqlalchemy
* https://zzun.app/repo/mfreeborn-fastapi-sqlalchemy-python-fastapi-utilities

In [ ]:
from fastapi import FastAPI
from fastapi_sqlalchemy import DBSessionMiddleware  # middleware helper
# an object to provide global access to a database session
from fastapi_sqlalchemy import db

from app.models import User

app = FastAPI()

app.add_middleware(DBSessionMiddleware, db_url="sqlite://")

# once the middleware is applied, any route can then access the database session
# from the global ``db``


@app.get("/users")
def get_users():
    users = db.session.query(User).all()

    return users


## Usage outside of a route
* schedulers example

In [ ]:
import pytz
# other schedulers are available
from apscheduler.schedulers.asyncio import AsyncIOScheduler
from fastapi import FastAPI
from fastapi_sqlalchemy import db

from app.models import User, UserCount

app = FastAPI()

app.add_middleware(DBSessionMiddleware, db_url="sqlite://")


@app.on_event('startup')
async def startup_event():
    scheduler = AsyncIOScheduler(timezone=pytz.utc)
    scheduler.start()
    # runs every night at midnight
    scheduler.add_job(count_users_task, "cron", hour=0)


def count_users_task():
    """Count the number of users in the database and save it into the user_counts table."""

    # we are outside of a request context, therefore we cannot rely on ``DBSessionMiddleware``
    # to create a database session for us. Instead, we can use the same ``db`` object and
    # use it as a context manager, like so:

    with db():
        user_count = db.session.query(User).count()

        db.session.add(UserCount(user_count))
        db.session.commit()

    # no longer able to access a database session once the db() context manager has ended

    return users


# run app using uvicorn

* 
```python
import uvicorn
uvicorn.run('main:app',host='127.0.0.1', port=9080, reload=True, access_log=False,workers=1, use_colors=True)
```
* api:app：这部分指定了要运行的 Python 模块和应用对象。在这个例子中，api 是 Python 文件（例如 api.py）的文件名，而 app 是该文件中包含 ASGI 应用的对象的名称。
* 或者：uvicorn api:app --host 0.0.0.0 --port 8000 --reload

# sync to async

* https://github.com/django/asgiref/tree/main

In [1]:
import asyncio
from fastapi import FastAPI
from asgiref.sync import async_to_sync

# app = FastAPI()

# # 异步的路径操作函数
# async def async_operation():
#     await asyncio.sleep(1)
#     return {"message": "Async operation completed"}

# # 使用 to_sync 转换异步函数为同步函数
# sync_operation = to_sync(async_operation)

# # 在路径上使用同步函数
# @app.get("/items/")
# def read_item():
#     result = sync_operation()
#     return result


In [2]:
import asyncio
from fastapi import FastAPI
from asgiref.sync import to_sync

app = FastAPI()

# 异步的路径操作函数
async def async_operation():
    await asyncio.sleep(1)
    return {"message": "Async operation completed"}

# 使用 to_sync 转换异步函数为同步函数
sync_operation = to_sync(async_operation)

# 在路径上使用同步函数
@app.get("/items/")
def read_item():
    result = sync_operation()
    return result


ImportError: cannot import name 'to_sync' from 'asgiref.sync' (c:\ProgramData\Anaconda3\lib\site-packages\asgiref-3.7.2-py3.9.egg\asgiref\sync.py)